In [11]:
!pip3 install folium
!pip3 install wget

In [13]:
import folium
import wget
import pandas as pd

In [14]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [15]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [17]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [18]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [19]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [20]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for idx,row in launch_sites_df.iterrows():
    circle = folium.Circle([row['Lat'],row['Long']], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site']))
    marker = folium.map.Marker(
        [row['Lat'],row['Long']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

In [21]:
# Task 2: Mark the success/failed launches for each site on the map
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [22]:
marker_cluster = MarkerCluster()

In [23]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [24]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, row in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.map.Marker(
        [row['Lat'],row['Long']],
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
        )
    marker_cluster.add_child(marker)

site_map

In [25]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df = spacex_df.assign(marker_color=lambda x:['green' if c==1 else 'red' for c in x['class']])
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [26]:
# TASK 3: Calculate the distances between a launch site to its proximities
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map


In [27]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [28]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [29]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
launch_site_lat=28.56326
launch_site_lon=-80.57684
coastline_lat=28.56401
coastline_lon=-80.56811
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.8569186317390074

In [30]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

distance_marker = folium.Marker(
   [coastline_lat,coastline_lon],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )
site_map.add_child(distance_marker)
site_map

In [31]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
lines=folium.PolyLine(locations=[[launch_site_lat,launch_site_lon],[coastline_lat,coastline_lon]], weight=1)
site_map.add_child(lines)
site_map

In [32]:
proximities_df = spacex_df.groupby('Launch Site').first()[['Lat','Long']].assign(Proximity=lambda x:['self' for i in range(len(x))]).set_index('Proximity', append=True)

proximities_df.loc[('CCAFS LC-40','coastline'),['Lat','Long']] = 28.5634,-80.56796
proximities_df.loc[('CCAFS LC-40','railway'),['Lat','Long']] = 28.57217,-80.58528
proximities_df.loc[('CCAFS LC-40','highway'),['Lat','Long']] = 28.56301,-80.57076
proximities_df.loc[('CCAFS LC-40','city'),['Lat','Long']] = 28.10469,-80.64784

proximities_df.loc[('CCAFS SLC-40','coastline'),['Lat','Long']] = 28.56409,-80.56806
proximities_df.loc[('CCAFS SLC-40','railway'),['Lat','Long']] = 28.57217,-80.58528
proximities_df.loc[('CCAFS SLC-40','highway'),['Lat','Long']] = 28.56385,-80.57088
proximities_df.loc[('CCAFS SLC-40','city'),['Lat','Long']] = 28.10469,-80.64784

proximities_df.loc[('KSC LC-39A','coastline'),['Lat','Long']] = 28.60283,-80.58815
proximities_df.loc[('KSC LC-39A','railway'),['Lat','Long']] = 28.57314,-80.65398
proximities_df.loc[('KSC LC-39A','highway'),['Lat','Long']] = 28.57307,-80.65553
proximities_df.loc[('KSC LC-39A','city'),['Lat','Long']] = 28.10469,-80.64784

proximities_df.loc[('VAFB SLC-4E','coastline'),['Lat','Long']] = 34.6347,-120.62531
proximities_df.loc[('VAFB SLC-4E','railway'),['Lat','Long']] = 34.63585,-120.62401
proximities_df.loc[('VAFB SLC-4E','highway'),['Lat','Long']] = 34.7048,-120.5694
proximities_df.loc[('VAFB SLC-4E','city'),['Lat','Long']] = 34.64253,-120.47331

proximities_df = proximities_df.query("Proximity == 'self'").reset_index("Proximity",drop=True).join(
    proximities_df.query("Proximity != 'self'").reset_index("Proximity", drop=False),
    lsuffix="LaunchSite"
).assign(
    distance=lambda x:[calculate_distance(row['LatLaunchSite'], row['LongLaunchSite'], row['Lat'], row['Long']) for idx,row in x.iterrows()]
).set_index("Proximity", append=True)
proximities_df


LatLaunchSite  LongLaunchSite       Lat       Long  \
Launch Site  Proximity                                                       
CCAFS LC-40  coastline      28.562302      -80.577356  28.56340  -80.56796   
             railway        28.562302      -80.577356  28.57217  -80.58528   
             highway        28.562302      -80.577356  28.56301  -80.57076   
             city           28.562302      -80.577356  28.10469  -80.64784   
CCAFS SLC-40 coastline      28.563197      -80.576820  28.56409  -80.56806   
             railway        28.563197      -80.576820  28.57217  -80.58528   
             highway        28.563197      -80.576820  28.56385  -80.57088   
             city           28.563197      -80.576820  28.10469  -80.64784   
KSC LC-39A   coastline      28.573255      -80.646895  28.60283  -80.58815   
             railway        28.573255      -80.646895  28.57314  -80.65398   
             highway        28.573255      -80.646895  28.57307  -80.65553   
             city           28.573255      -80.646895  28.10469  -80.64784   
VAFB SLC-4E  coastline      34.632834     -120.610745  34.63470 -120.62531   
             railway        34.632834     -120.610745  34.63585 -120.62401   
             highway        34.632834     -120.610745  34.70480 -120.56940   
             city           34.632834     -120.610745  34.64253 -120.47331   

                         distance  
Launch Site  Proximity             
CCAFS LC-40  coastline   0.926054  
             railway     1.343093  
             highway     0.649222  
             city       51.365738  
CCAFS SLC-40 coastline   0.861525  
             railway     1.295798  
             highway     0.584818  
             city       51.471476  
KSC LC-39A   coastline   6.613767  
             railway     0.692172  
             highway     0.843713  
             city       52.118441  
VAFB SLC-4E  coastline   1.349004  
             railway     1.259455  
             highway     8.853368  
             city       12.623666

In [33]:
proximities_df.groupby(level='Proximity')['distance'].mean()

Proximity
city         41.894830
coastline     2.437587
highway       2.732780
railway       1.147629
Name: distance, dtype: float64

In [34]:
for idx,row in proximities_df.iterrows():
        
    distance_marker = folium.Marker(
        [row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(row['distance']),
        )
    )
    site_map.add_child(distance_marker)
    
    lines=folium.PolyLine(locations=[[row['LatLaunchSite'], row['LongLaunchSite']], [row['Lat'], row['Long']]], weight=1)
    site_map.add_child(lines)
site_map